In [1]:
!pip install torch_geometric
# !pip install torch_sparse

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.4 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import json
import scipy.sparse as sp

In [8]:
# Step 1: Load the dataset

adjacency_matrix = np.load('adj.npz')
feature_matrix = np.load('features.npy')
labels = np.load('labels.npy')
with open('splits.json', 'r') as f:
  data_splits = json.load(f)


data, indices, indptr, shape = adjacency_matrix['data'], adjacency_matrix['indices'], adjacency_matrix['indptr'], adjacency_matrix['shape']
adjacency_matrix = sp.csr_matrix((data, indices, indptr), shape=shape)
coo_matrix = adjacency_matrix.tocoo()

# Extract edge indices
edge_index = np.vstack((coo_matrix.row, coo_matrix.col))


train_idx = data_splits['idx_train']

#if not in the train, put in the test
test_idx = []
for i in range(2480):
  if i not in train_idx:
    test_idx.append(i)


train_feature_matrix = feature_matrix[train_idx]
test_feature_matrix = feature_matrix[test_idx]

data, indices, indptr, shape, edge_index

#1390 x 1984 for test, #1390 x 496 for train
# print(np.size(test_feature_matrix, axis=0))
# print(np.size(test_feature_matrix, axis=0))
# print(np.size(train_feature_matrix, axis=1))
# print(np.size(train_feature_matrix, axis=1))

(array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
 array([1084, 1104, 1288, ...,  931,  933,  999], dtype=int32),
 array([    0,     5,     9, ..., 10095, 10099, 10100], dtype=int32),
 array([2480, 2480]),
 array([[   0,    0,    0, ..., 2478, 2478, 2479],
        [1084, 1104, 1288, ...,  931,  933,  999]], dtype=int32))

In [9]:
# Create PyTorch Geometric Data object for training
x_train = torch.tensor(train_feature_matrix, dtype=torch.float)
y_train = torch.tensor(labels, dtype=torch.long)
train_data = Data(x=x_train, edge_index=edge_index, edge_attr=None, y=y_train)

In [10]:
# Create PyTorch Geometric Data object for testing
x_test = torch.tensor(test_feature_matrix, dtype=torch.float)
test_data = Data(x=x_test, edge_index=edge_index, edge_attr=None)

test_data.is_directed()

TypeError: sort() got an unexpected keyword argument 'stable'

In [11]:
# Define Graph Neural Network model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(feature_matrix.shape[1], 16)
        self.conv2 = GCNConv(16, len(np.unique(labels)))

    def forward(self, data):
        x = data.x
        x = self.conv1(x, data.edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [7]:
# Instantiate the model, optimizer, and loss function
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()

# Train the model
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(train_data)
    loss = criterion(out, train_data.y)
    loss.backward()
    optimizer.step()

NameError: name 'feature_matrix' is not defined